In [1]:
import numpy as np
import pandas as pd
from helpfiles.temp_save_load import save_files, load_files
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import helpfiles.ml_pipeline_config as configurations

#Load data
df = pd.read_csv("C:/Users/PC Paul/Downloads/DataEngOwnProject/data/creditcard.csv")#.head(10000) 
while len(df.index) < 1000000:
    df = pd.concat([df,df])
df.reset_index(drop = True, inplace = True)

#scale_data():
rob_scaler = RobustScaler()
df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1,1))
df.drop(['Time','Amount'], axis=1, inplace=True)


#split_data_train_val():
split_ratio = configurations.params['test_split_ratio']
# Splitting
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
# Dataframe is concatenated again as the full dataset is needed and split again during cross validation. _val dataframes are needed for the final evaluation. 
df = pd.concat([X_train, y_train],axis = 1)

#undersample_data():
df2 = df.sample(frac=1) # this shuffles the initial df
# Filter fraud cases and concat equal amount of non-fraud-cases
fraud_df = df2.loc[df['Class'] == 1]
non_fraud_df = df2.loc[df['Class'] == 0][:len(fraud_df)]
normal_distributed_df = pd.concat([fraud_df, non_fraud_df])
# Shuffle dataframe rows
new_df = normal_distributed_df.sample(frac=1, random_state=42)

test_observation = new_df.iloc[4:5, :-1].values
test_observation

import joblib
svc.predict(test_observation)


now = datetime.now().strftime('%d-%m-%Y_%H-%M-%S')
filename = 'model_' + '.pkl'
joblib.dump(svc, filename, compress=1)
mj = joblib.load(filename)
mj.predict(test_observation)

In [28]:
#from sklearn.decomposition import PCA, TruncatedSVD

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *
import collections

# Other Libraries
#from sklearn.model_selection import train_test_split
#from sklearn.pipeline import make_pipeline
#from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
#from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import NearMiss
#from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

import helpfiles.ml_pipeline_config as configurations

#experiment():
X = new_df.drop('Class', axis=1)
y = new_df['Class']





# Logistic Regression 
log_reg_maxiter = configurations.params["logreg_maxiter"]
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} #todo: put this into pipeline config
# Grid search
grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params, scoring = 'f1')
#grid_log_reg = GridSearchCV(LogisticRegression(max_iter = log_reg_maxiter), log_reg_params) # todo: see whether you can use this line
grid_log_reg.fit(X, y)
# Logistic Regression estimator:
log_reg = grid_log_reg.best_estimator_

# Support Vector Classifier
svc_params = {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']} #todo: put this into pipeline config
# Grid search
grid_svc = GridSearchCV(SVC(), svc_params, scoring = 'f1')
grid_svc.fit(X, y)
# SVC best estimator
svc = grid_svc.best_estimator_

# Evaluation
# Logistic Regression 
log_reg_score = cross_val_score(log_reg, X, y, cv=5, scoring = 'f1')
print('Logistic Regression Cross Validation F1 score: ', round(log_reg_score.mean() * 100, 2).astype(str) + '%')
print("Best Parameters Logistic Regression: " + str(grid_log_reg.best_params_))
# SVC
svc_score = cross_val_score(svc, X, y, cv=5, scoring = 'f1')
print('Support Vector Classifier Cross Validation F1 score', round(svc_score.mean() * 100, 2).astype(str) + '%')
print("Best Parameters SVM: " + str(grid_svc.best_params_))

#X_val = load_files(['X_val'])[0]
#y_val = load_files(['y_val'])[0]

#Predict Y
y_pred_log_reg = log_reg.predict(X_val)
y_pred_SVM = svc.predict(X_val)
f1_log_reg = f1_score(y_val, y_pred_log_reg)
f1_SVM = f1_score(y_val, y_pred_SVM)

#******************************************************************************************************

# Initialize the estimators
log_reg_maxiter = configurations.params["logreg_maxiter"]

clf1 = SVC()
clf2 = LogisticRegression(max_iter = log_reg_maxiter)
clf3 = KNeighborsClassifier()
clf4 = GradientBoostingClassifier()

# Initialize hyperparameters for each dictionary

param1 = {}
param1['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param1['classifier__kernel'] = ['rbf', 'poly', 'sigmoid', 'linear']
#param1['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param1['classifier'] = [clf1]

param2 = {}
param2['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param2['classifier__penalty'] = ['l1', 'l2']
#param2['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param2['classifier'] = [clf2]

param3 = {}
param3['classifier__n_neighbors'] = [2,5,10,25,50]
param3['classifier'] = [clf3]

param4 = {}
param4['classifier__n_estimators'] = [10, 50, 100, 250]
param4['classifier__max_depth'] = [5, 10, 20]
param4['classifier'] = [clf4]



pipeline = Pipeline([('classifier', clf1)])
params = [param1, param2, param3, param4]

# Train the grid search model
gs = GridSearchCV(pipeline, params, cv=5, n_jobs=-1, scoring='recall').fit(X,y)
print(gs.best_params_)

# Predict Y
y_pred= gs.best_estimator_.predict(X_val)

# Calculate recall
recall_val = recall_score(y_val, y_pred)
print("Recall validation result:" + str(recall_val))

#******************************************************************************************************




now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") #todo: globalize time stamp

log_reg_results = pd.DataFrame([[
    now,
    'Logistic Regression',
    grid_log_reg.best_estimator_,
    grid_log_reg.best_params_,
    f1_log_reg
    ]],
    columns = [
        'experiment_date',
        'method',
        'best_estimator',
        'best_parameters',
        'f1_score'])

svc_results = pd.DataFrame([[
    now,
    'Support Vector Classifier',
    grid_svc.best_estimator_,
    grid_svc.best_params_,
    f1_SVM
    ]],
    columns = [
        'experiment_date',
        'method',
        'best_estimator',
        'best_parameters',
        'f1_score'])

results2 = pd.concat([log_reg_results,svc_results])

#******************************************************************************************************
#Creation of results table 
results = pd.DataFrame([[
    now,
    gs.best_params_['classifier'],
    recall_val
    ]],
    columns = [
        'experiment_date',
        'best_estimator',
        'recall_score'])

#Creation of the dataset of evalutation metrics
y_val_series = pd.Series(y_val)
y_pred_series = pd.Series(y_pred)

target_prediction = pd.concat([y_val.reset_index(),y_pred_series], axis = 1, ignore_index=True)
target_prediction[0] = now
target_prediction.columns = [
    'experiment_date',
    'actual_class',
    'predicted_class']

# Todo: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADD save target prediction

Logistic Regression Cross Validation F1 score:  94.57%
Best Parameters Logistic Regression: {'C': 10, 'penalty': 'l2'}
Support Vector Classifier Cross Validation F1 score 94.79%
Best Parameters SVM: {'C': 0.9, 'kernel': 'linear'}
{'classifier': GradientBoostingClassifier(max_depth=10, n_estimators=50), 'classifier__max_depth': 10, 'classifier__n_estimators': 50}
Recall validation result:0.9867549668874173


In [61]:
y_val_series = pd.Series(y_val)
y_pred_series = pd.Series(y_pred)

target_prediction = pd.concat([y_val.reset_index(),y_pred_series], axis = 1, ignore_index=True)
target_prediction[0] = now
target_prediction.columns = ['experiment_date','actual_class','predicted_class']

#target_prediction = pd.DataFrame([[
#    now,
#    y_val,
#    y_pred
#    ]],
#    columns = [
#        'experiment_date',
#        'actual_class',
#        'predicted_class'])

target_prediction

,experiment_date,actual_class,predicted_class
0,2022-03-10_20-28-17,0,0
1,2022-03-10_20-28-17,0,0
2,2022-03-10_20-28-17,0,0
3,2022-03-10_20-28-17,0,0
4,2022-03-10_20-28-17,0,0
...,...,...,...
341764,2022-03-10_20-28-17,0,0
341765,2022-03-10_20-28-17,0,0
341766,2022-03-10_20-28-17,0,0
341767,2022-03-10_20-28-17,0,0


In [54]:
y_val


1078193    0
1036416    0
567866     0
942365     0
638911     0
          ..
281795     0
945664     0
1020521    0
30826      0
1079156    0
Name: Class, Length: 341769, dtype: int64

In [34]:
recall_val = recall_score(y_val, y_pred)
print("recall validation result: " + str(recall_val))

precision_val = precision_score(y_val, y_pred)
print("precision validation result: " + str(precision_val))

recall validation result: 0.9867549668874173
precision validation result: 0.05136159944846604


# Second try as a pipeline

In [8]:
# Import necessary packages
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *

In [9]:
# Initialize the estimators
clf1 = SVC()
clf2 = LogisticRegression()
clf3 = KNeighborsClassifier()
clf4 = GradientBoostingClassifier()



# Initialize hyperparameters for each dictionary

param1 = {}
param1['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param1['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param1['classifier'] = [clf1]

param2 = {}
param2['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param2['classifier__penalty'] = ['l1', 'l2']
param2['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param2['classifier'] = [clf2]

param3 = {}
param3['classifier__n_neighbors'] = [2,5,10,25,50]
param3['classifier'] = [clf3]

param4 = {}
param4['classifier__n_estimators'] = [10, 50, 100, 250]
param4['classifier__max_depth'] = [5, 10, 20]
param4['classifier'] = [clf4]



pipeline = Pipeline([('classifier', clf1)])
params = [param1, param2, param3, param4]



In [10]:


%%time
# Train the grid search model
gs = GridSearchCV(pipeline, params, cv=3, n_jobs=-1, scoring='roc_auc').fit(X,y)



CPU times: total: 8.69 s
Wall time: 44.6 s


In [11]:
gs.best_params_

{'classifier': GradientBoostingClassifier(max_depth=5, n_estimators=250),
 'classifier__max_depth': 5,
 'classifier__n_estimators': 250}

In [12]:
score_f1 = cross_val_score(gs.best_estimator_, X, y, cv=5, scoring = 'f1')
score_f1.mean()

0.9872870137837874

In [13]:
#Predict Y
y_pred= gs.best_estimator_.predict(X_val)
f1_val = f1_score(y_val, y_pred)
f1_val

0.25371527029937535